In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import f1_score
import seaborn as sns

In [ ]:
df=pd.read_csv("../input/texephyr/train_brain.csv")
test=pd.read_csv('../input/texephyr/test_brain.csv')

In [ ]:
df["nancount"]=df.isnull().sum(axis=1)
test['nancount']=test.isnull().sum(axis=1)

In [ ]:
df["test"]=0
test["test"]=1
total=pd.concat([df,test],axis=0)

In [ ]:
total["target"].dtype

In [ ]:
for i in total.select_dtypes("object"):
    lb=LabelEncoder()
    total[i]=lb.fit_transform(total[i])
    df[i]=lb.transform(df[i])

In [ ]:
for i in total.select_dtypes("object"):
    lb=LabelEncoder()
    total[i]=lb.fit_transform(total[i])
   # df[i]=lb.transform(df[i])

In [ ]:
#train=pd.concat([total[total["target"]==0].sample(25000,replace=True),total[total["target"]==1]],axis=0).copy()
train=total[total["test"]==0].copy()
test=total[total["test"]==1].copy()
#.drop(["target","test"],axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( train.drop(["target","test"],axis=1), train["target"], test_size=0.33, random_state=42)

In [ ]:

lgb=LGBMClassifier(random_state=20)
lgb2=LGBMClassifier(random_state=20)

lgb.fit(X_train,y_train)
pre1=lgb.predict(X_train)
lgb2.fit(pd.concat([X_train,pd.DataFrame({"pre":pre1}).set_index(X_train.index)],axis=1),y_train)


In [ ]:
pre=lgb.predict(X_test)
final_pred=lgb2.predict(pd.concat([X_test,pd.DataFrame({"pre":pre}).set_index(X_test.index)],axis=1))

In [ ]:
f1_score(y_test,final_pred,average="weighted")
##Stacked Performing Well Lets try one submission

In [ ]:
## Concatanated LGBM Architecture

In [ ]:
lgb=LGBMClassifier(random_state=20)
lgb2=LGBMClassifier(random_state=20)

lgb.fit(train.drop(["target","test"],axis=1),train["target"])
pre1=lgb.predict(train.drop(["target","test"],axis=1))
lgb2.fit(pd.concat([train.drop(["target","test"],axis=1),pd.DataFrame({"pre":pre1}).set_index(train.index)],axis=1),train["target"])


In [ ]:
test

In [ ]:
pre=lgb.predict(test.drop(["target","test"],axis=1))
final_pred=lgb2.predict(pd.concat([test.drop(["target","test"],axis=1),pd.DataFrame({"pre":pre}).set_index(test.index)],axis=1))

In [ ]:
pd.Series(final_pred).value_counts()

In [ ]:
stacklgb=pd.DataFrame({0:pd.Series(final_pred)})

In [ ]:
stacklgb.to_csv("stacklgb.csv",index=False)

In [ ]:
# Finally In Excel ,to make the submission according to the sample submission (I removed column name :0) and shifted all rows up.
#That will be my final Submission which lead to my second highest score .0.7361